In [221]:
import pandas as pd
from tqdm import tqdm
import json
import string
from typing import List
from time import sleep

In [205]:
import openai
openai.api_key = ""

# with open("./request_data_full_w_cands.json", "r") as f:
#     request_data = json.load(f)
with open("./request_data_full_w_inf_cands.json", "r") as f:
    request_data = json.load(f)

with open("./system_prompt.txt", "r") as f:
    system_prompt = f.read().strip()

with open("./informal_attrs.txt", "r") as f:
    informal_attrs = f.read().strip()

with open("./formal_attrs.txt", "r") as f:
    formal_attrs = f.read().strip()

request_data["messages"][0]['content'] = system_prompt + "\n\n" + informal_attrs + "\n\n" + formal_attrs
print(request_data["messages"][0]['content'])
request_data

You are an expert forensic linguist. Your task is to identify infromal attributes in a setnece, modify them to create a formal sentence, and then output the attributes of the generated formal sentence. Use the following format: attribute (excertp from text in quotation marks). Make sure to provide a complete list of informal and formal atttributes. Focus on what has changed between formal and informal sentences.

Here is a list of attributes that can help identify an informal sentence:
Contractions: Use of contractions like "I'm," "can't," "won't."
Abbreviations and textese: Use of abbreviations and shortened words like "u," "lol," "brb."
Slang and colloquialisms:
   - Informal: "Hey, what's up?"
   - Formal: "Hello, how are you?"
Simple sentence structures: Shorter and less complex sentences.
   - Informal: "I went to the store. I bought milk."
   - Formal: "I went to the store and purchased milk."
Informal punctuation: Frequent use of exclamation marks, ellipses, dashes, etc.
   - In

{'messages': [{'role': 'system',
   'content': 'You are an expert forensic linguist. Your task is to identify infromal attributes in a setnece, modify them to create a formal sentence, and then output the attributes of the generated formal sentence. Use the following format: attribute (excertp from text in quotation marks). Make sure to provide a complete list of informal and formal atttributes. Focus on what has changed between formal and informal sentences.\n\nHere is a list of attributes that can help identify an informal sentence:\nContractions: Use of contractions like "I\'m," "can\'t," "won\'t."\nAbbreviations and textese: Use of abbreviations and shortened words like "u," "lol," "brb."\nSlang and colloquialisms:\n   - Informal: "Hey, what\'s up?"\n   - Formal: "Hello, how are you?"\nSimple sentence structures: Shorter and less complex sentences.\n   - Informal: "I went to the store. I bought milk."\n   - Formal: "I went to the store and purchased milk."\nInformal punctuation: Fr

In [206]:
def get_cands(informal_sentence, formal_sentence):
  """Obtain candidates for formal and informal sentences."""
  formal_words = set(formal_sentence.translate(str.maketrans("", "", string.punctuation))
                      .split()
                      )
  informal_words = set(informal_sentence.translate(str.maketrans("", "", string.punctuation))
                      .split()
                      )
  # print(formal_words, informal_words)
  #set_difference_formal_informal = formal_words - informal_words
  set_difference_informal_formal = informal_words - formal_words

  return set_difference_informal_formal #, set_difference_formal_informal

def gen_expls(request_data, informal, formal, model="gpt-3.5-turbo"):
  """Generate explanations for a given informal, formal sentence pair."""

  inf_cands = get_cands(informal, formal)

  request_data['messages'][-1]['content'] = f"Informal: {informal}\nInformal Candidates: {inf_cands}"
  # print(request_data)
  # assert False
  response = openai.ChatCompletion.create(
      model=model,
      messages=request_data["messages"]
  )
  # print(response)
  # print("*"*20)
  result = ''
  for choice in response.choices:
      result += choice.message.content
  return result

# print(gen_expls(request_data, "Sure, it's ok, but I always have let the guy ask me.", 
# "I prefer to let the guy ask me.",
# request_data))

In [214]:
def gen_data(request_data, start: int, end: int) -> List[List[str]]:
  """Generates data from the GYAFC dataset."""

  data_folder = "../data/GYAFC_Corpus/Family_Relationships/train"

  with open(f"{data_folder}/informal", "r") as f:
    inform = f.readlines()
    #print(len(inf))

  with open(f"{data_folder}/formal", "r") as f:
    form = f.readlines()
    #print(len(form))

  data = []
  for i, f in tqdm(zip(inform[start:end], form[start:end])):
  #for i, f in zip(inform[start:end], form[start:end]):
    informal_sentence, formal_sentence = i.strip(), f.strip()
    response = gen_expls(request_data, i, f)
    data.append([informal_sentence, formal_sentence, response])
  return data

#data = gen_data(request_data, 0, 100)

In [208]:
def format_res(input_string):
    try:
        split_string = input_string.split('\n')
        informal_attributes = split_string[0].split("Informal Style: ")[1].strip()
        formal_description = split_string[1].split("Formal: ")[1].strip()
        formal_cands = split_string[2].split("Formal Candidates: ")[1].strip()
        formal_attributes = split_string[3].split("Formal Style: ")[1].strip().rstrip('.')

        return informal_attributes, formal_description, formal_attributes
    except:
        return None, None, None

In [ ]:
batch_size = 100
for i in range(19010, 26000, batch_size):
  print(f"Generating data for {i} to {i+batch_size}")
  try:
    data = gen_data(request_data, i, i+batch_size)
  except:
    sleep(60)
    data = gen_data(request_data, i, i+batch_size)
  df = pd.DataFrame(data, columns=['informal', 'formal', 'expl'])
  # add columns returned by the function format_res to df
  df[['informal_attributes', 
  'formal_description', 
  'formal_attributes']] = df['expl'].apply(lambda x: pd.Series(format_res(x)))
  df.to_csv(f"../data/gyafc_expls/gpt35_expls_{i}_{i+batch_size}.csv", index=False)